# Getting Bsky Data
Sara Rosenau 2/21/2025  
This notebook will feature me trying to learn the Bsky API and collect some data from it

## Playing around with the api a bit
- First, import the api (`atproto`) and start a new client session

In [1]:
from atproto import Client, client_utils

In [3]:
client = Client()
client.login('sararosenauling.bsky.social', 'fjx5-6tuy-oks4-igsn')

ProfileViewDetailed(did='did:plc:66lizgbz577cdzpwnfyox3g6', handle='sararosenauling.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:66lizgbz577cdzpwnfyox3g6/bafkreicddmiczeju5vfi5yxgp2oosshkakweumupixgkofwwkq6jp3nqiq@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:66lizgbz577cdzpwnfyox3g6/bafkreih3qtqbp27r3sq6jskcn4wmb7oxmbbbt6p3lcmhd5zbqi6ww5k5pm@jpeg', created_at='2023-06-25T12:38:14.621Z', description='she/her - Sociolinguist studying the internet at Pitt! Also into kpop/kdrama, musicals, the Phillies, cross stiching and crocheting, and singing in general\n', display_name='Sara Rosenau', followers_count=893, follows_count=1051, indexed_at='2024-11-11T20:50:23.741Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=297, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=N

### Making a post from the api!

In [5]:
#client.send_post(text='testing api')

CreateRecordResponse(uri='at://did:plc:66lizgbz577cdzpwnfyox3g6/app.bsky.feed.post/3lipnaaejya2u', cid='bafyreidr4z47xkshfa4ej2ychc2oskymyvkz6zlzgi6ielbhzdip75ff2a')

### Getting posts from my timeline
- Following [this example](https://github.com/MarshalX/atproto/blob/main/examples/home_timeline.py)
- Getting 10 posts from my following feed

In [8]:
def main() -> None:
    print('Home (Following):\n')

    timeline = client.get_timeline(limit=10, algorithm='reverse-chronological')
    for feed_view in timeline.feed:
        action = 'New Post'
        if feed_view.reason:
            action_by = feed_view.reason.by.handle
            action = f'Reposted by @{action_by}'

        post = feed_view.post.record
        author = feed_view.post.author

        print(f'[{action}] {author.display_name}: {post.text}')

In [9]:
main()

Home (Following):

[New Post] Dr. Sarah Parcak : They are not your target. Trust me. I know you are angry. Call your congresspeople. Screaming here doesn't do a damn thing.
[New Post] jenny_tightpants🪑: free him
[Reposted by @logophobe.bsky.social] Inclusive Diversity Pronouns at the Woke NSA: Every liberal "paper of record": maybe the fascists have a point 
Eater: Know your rights
[Reposted by @prisonculture.bsky.social] Inclusive Diversity Pronouns at the Woke NSA: Every liberal "paper of record": maybe the fascists have a point 
Eater: Know your rights
[Reposted by @swordsjew.bsky.social] CJ Ciaramella: I am obligated to note that the Fort Bliss camp already exists and was used by the Biden admin to hold migrant children and teenagers. In 2021 I got whistleblower recordings of officials discussing filthy conditions and lack of medical care: reason.com/2021/05/24/s...
[Reposted by @prisonculture.bsky.social] People's Book: That's right! Come on by for Fed Fridays - have a cup of joe 